# People Matching Algortihm

## Steps to be done

1. Prepare and clean the datasets
2. Calculate similarity within the two user groups
3. Assign mentor to each mentee based on similarity
3. Filter the assignments by the following preferences:
    * Mentee's priority nr 1: theatre, role, segment or gender
    * Mentor's prefered theatre
    * Mentor's prefered role
    * Mentor's prefered gender
4. Filter by mentor, if he/she has a mentee in mind
5. Shuffle the assignments until every mentee has a unique mentor assigned (shuffle top 4 mentors)
6. Exporting to excel sheet by ranking


# 1. Prepare and clean the datasets
## 1.1. Mentor Data

In [672]:
import pandas as pd
import scipy.spatial
import random
import xlsxwriter
import warnings
warnings.simplefilter("ignore")

In [673]:
# reed in mentor data
df_mentor_raw = pd.read_excel(r"FY21 - Chairman Mentor Interest Form.xlsx",sheet_name="Form_a")

# check out data
df_mentor_raw.shape  #(91, 41)

(85, 40)

In [674]:
# function that drops a column if it contains only NaN values
def drop_nan_cols(df):
    nr = df.shape[0]
    for column in df.columns:
        if df[column].isnull().sum(axis = 0) == nr:
            df.drop([column], axis=1, inplace=True)
    return df

df_mentor_raw = drop_nan_cols(df_mentor_raw)
df_mentor_raw.shape  #(91, 29) removed 12 cols full of NaNs

(85, 28)

In [675]:
# remove first two entries since these were test entries
# df_mentor_raw.drop([0,1], axis=0, inplace=True)

# replace the numbered index by mentor names
df_mentor_raw = pd.DataFrame(df_mentor_raw).set_index('Name', drop=True)

In [676]:
df_mentor_raw.head()

,ID,Start time,Completion time,How many Y2 CSAP Associates are you willing to mentor?,Gender,Are you a CSAP Alum?2,What is your CSAP Class FY?,Which track were you in?,Have you mentored an Early in Career before?,What are you most likely to give in this mentoring engagement?,...,Your segment?,What was most challenging about the engagement and how did you overcome it?,Your country of residence,What are your general strengths?,What are your personal interests?,What do you think qualifies you to be a good mentor?2,What are your early expectations from this mentoring engagement?,Have you already mentored for the Chairman Club Mentoring Program?,Any final comments or suggestions before you submit?,Let us know if you already have a CSAP mentee in mind
Name,,,,,,,,,,,,,,,,,,,,,
22341,57,44175.679884,44175.684433,1,M,No,NaN,NaN,No,I'm flexible and will adapt to my mentee's goals;,...,Service Provider,NaN,Brazil,Design thinking;Cross-architecture;Story telling;,"Team sports (soccer, football);Travel;Music;","Experience, respect, listening, generosity, ho...",Help the mentee accelerate his/her development...,No,NaN,NaN
22342,67,44179.689433,44179.700625,2,F,No,NaN,NaN,Yes,I'm flexible and will adapt to my mentee's goals;,...,CX,Skill development through opportunities for le...,USA,Customer interaction;Influencing;Team building;,Cooking;Pets;Travel;,I'm motivated through my contributions to help...,Help provide insight and guidance from a CX pe...,No,Looking forward to seeing if I can help :),NaN
22343,31,44172.801921,44172.804630,1,F,No,NaN,NaN,No,Sharing my own career journey and lessons lear...,...,SLED,NaN,USA,Accountability;Collaboration;Communications (p...,Cooking;Culture;Design;Gardening;Individual sp...,"Candid feedback, honesty and the tools to prov...","Providing guidance, support and tools for the ...",No,Thank you for the opportunity.,NaN
22344,87,44182.376516,44183.028275,2,M,Yes,FY09,ASE,No,I'm flexible and will adapt to my mentee's goals;,...,Enterprise,NaN,Australia,Influencing;Customer interaction;Competitive s...,"Family;Team sports (soccer, football);Travel;",I,To both provide the guidance that was afforded...,No,NaN,NaN
22345,46,44173.561331,44173.566944,2,F,No,NaN,NaN,Yes,General strategic career advice depending on m...,...,Partner,Time frame \nAnd over came it by being persist...,UAE,Negotiating;Personal Brand;Team building;Criti...,Art;Family;Volunteering;Reading;Music;,Knowledge and skills \nAbility to see the best...,Develop and progress \nExpand my knowledge \nG...,No,NaN,NaN


Up to this point, the dataset is clean from data we do not need for matching. 

In [677]:
type(df_mentor_raw)

pandas.core.frame.DataFrame

In [678]:
#df_mentor_raw.isnull().sum(axis = 0)

In [679]:
df_mentor = df_mentor_raw[["What are you most likely to give in this mentoring engagement?",
                           "What are your general strengths?",
                           "What are your personal interests?"]]

#df_mentor.head()

In [680]:
# function that splits the choices in the colums by ; to and adds them to a list
def extract_choices(df_col):
    choices = []
    cols = set()
    data = set(df_col)
    for choice in data:
        choice = choice.split(";")
        choices.append(choice)
    for innerlist in choices:
        for choice in innerlist:
            if choice != "":
                cols.add(choice)    
        
    return list(cols)

give_mentor = extract_choices(df_mentor["What are you most likely to give in this mentoring engagement?"])
strengths_mentor = extract_choices(df_mentor["What are your general strengths?"])
interests_mentor = extract_choices(df_mentor["What are your personal interests?"])

In [681]:
# make one hot df

def encode_onehot(df_input, df_output):
    count = 0
    for answer in df_output.columns:
        for column in df_input.columns:
            for result in df_input[column]:
                if answer in result: 
                    df_output.loc[df_output.index[count], answer] = 1           #.iloc[:,2:3]               
                                       
                if count == (len(df_input.index)-1):
                    count = 0
                else:
                    count += 1
    df_output = df_output.fillna(0)
    return df_output

### 1.1.1. Mentor Question 1: What are you most likely to give in this mentoring engagement?

This section creates a onehot encoded dataframe of the answers given by the mentors to this question.

The responses were classifies into 4 main types: 
* **General Advice**
* **Specific Advice**
* **Day to Day Advice**
* **Own Advice**

The following responses were edited, due to their multiple meanings:
* **All or any of the above** --> matches all the 4 response types
* **I'm flexible and will adapt to my mentee's goals** --> matches all the 4 response types

The following responses (recorded in the "others" field) were merged as following:
* **Contact Center knowledge transfer** -->  matches *Specific Advice*
* **how to become a trusted advisor and support customer with strategic planning** --> matches *Own Advice*
* **Sharing my own career journey and lessons learnt** --> matches *Own Advice*
* **An out of the box angle back on a situation.** --> matches *Own Advice*

In [682]:
df_give_mentor = encode_onehot(df_mentor, pd.DataFrame([], index=df_mentor.index, columns=give_mentor))

In [683]:
df_give_mentor.head()

,I'm flexible and will adapt to my mentee's goals,an out of the box angle back on a situation.,General strategic career advice depending on mentee goals,Sharing my own career journey and lessons learnt,how to become a trusted advisor and support customer with strategic planning,Specific career advice to prepare for next role,Tactical day to day situations & problem solving,all or any of the above
Name,,,,,,,,
22341,1,0,0,0,0,0,0,0
22342,1,0,0,0,0,0,0,0
22343,1,0,0,1,0,0,0,0
22344,1,0,0,0,0,0,0,0
22345,1,0,1,0,0,0,0,0


In [684]:
# function to combine multiple onehot encoded columns into one and optionally adding to new df
def combine_cols(df, combine_cols, add_to_df="no", combine_name=None):
    for col in combine_cols:
        if col == combine_cols[0]:
            try:
                result_list = list(df[col])
            except:
                result_list = [0] * len(df)
        else:
            try:
                result_list = [x + y for x, y in zip(result_list, list(df[col]))]
            except:
                continue
    result_list = [1 if x>1 else x for x in result_list]
            
    if type(add_to_df) == pd.core.frame.DataFrame:
        add_to_df[combine_name] = result_list
        add_to_df.set_index(df.index.values, inplace=True)
        return add_to_df
    else:
        return result_list

In [685]:
df_give_mentor_final = pd.DataFrame()

# combining the columns
cols = ["General strategic career advice depending on mentee goals", 
        "all or any of the above", 
        "I'm flexible and will adapt to my mentee's goals"]
df_give_mentor_final = combine_cols(df_give_mentor, cols, add_to_df=df_give_mentor_final, combine_name="general_advice")

# Specific advice
cols = ["all or any of the above", # own answer
        "I'm flexible and will adapt to my mentee's goals",
        "Specific career advice to prepare for next role"]
df_give_mentor_final = combine_cols(df_give_mentor, cols, add_to_df=df_give_mentor_final, combine_name="specific_advice")

# Advice of day to day situations & problem solving
cols = ["Tactical day to day situations & problem solving", 
        "all or any of the above", 
        "I'm flexible and will adapt to my mentee's goals"]
df_give_mentor_final = combine_cols(df_give_mentor, cols, add_to_df=df_give_mentor_final, combine_name="day2day_advice")

# Own experience
cols = ["how to become a trusted advisor and support customer with strategic planning ", # own answer
        "Sharing my own career journey and lessons learnt", 
        "an out of the box angle back on a situation. ", # own answer
        "all or any of the above", 
        "I'm flexible and will adapt to my mentee's goals"]
df_give_mentor_final = combine_cols(df_give_mentor, cols, add_to_df=df_give_mentor_final, combine_name="own_advice")


This is the final one hot encoded dataframe of the answers to Mentor Question 1:

In [686]:
df_give_mentor_final.head()

,general_advice,specific_advice,day2day_advice,own_advice
22341,1,1,1,1
22342,1,1,1,1
22343,1,1,1,1
22344,1,1,1,1
22345,1,1,1,1


### 1.1.2. Mentor Question 2: What are you general strengths?

This section creates a onehot encoded dataframe of the answers given by the mentors to this question.

The following responses (recorded in the "others" field) were merged as following:
* **Social Media** -->  matches *Creativity/Innovation*
* **Rapport Building** --> matches *Customer Interaction* and *Crucial Conversations*
* **Team building** --> matches *Empowerment* and *Recognition*
* **Collaboration** --> matches *Productivity*
* **Accountability** --> matches *Recognition*

In [687]:
df_strengths_mentor = encode_onehot(df_mentor, pd.DataFrame([], index=df_mentor.index, columns=strengths_mentor))

In [688]:
df_strengths_mentor.head()

,Competitive strategy,Decision making,Influencing,Creativity / innovation,Collaboration,Cross-architecture,Effective listening,Story telling,Rapport Building,Critical thinking,...,Providing feedback,Team building,Communications (public speaking),Coaching,Crucial conversations,Social Media,Delegating,Finance,Productivity,Goal setting
Name,,,,,,,,,,,,,,,,,,,,,
22341,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
22342,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
22343,0,1,0,1,1,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1
22344,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22345,0,0,0,1,0,0,0,0,0,1,...,0,1,0,1,0,0,0,0,0,0


In [689]:
df_strengths_mentor_final = pd.DataFrame()
exclude = []

# Customer interaction & Rapport Building
cols = ["Customer interaction", 
        "Rapport Building"] # own answer
df_strengths_mentor_final = combine_cols(df_strengths_mentor, cols, add_to_df=df_strengths_mentor_final, combine_name="Customer interaction")
exclude.append(cols)

# Crucial Conversations & Rapport Building
cols = ["Crucial conversations", 
        "Rapport Building"] # own answer
df_strengths_mentor_final = combine_cols(df_strengths_mentor, cols, add_to_df=df_strengths_mentor_final, combine_name="Crucial conversations")
exclude.append(cols)

exclude = [item for sublist in exclude for item in sublist]

In [690]:
for column in df_strengths_mentor.columns: 
    if column not in exclude:
        df_strengths_mentor_final[column] = list(df_strengths_mentor[column])

In [691]:
df_strengths_mentor_final.head()

,Customer interaction,Crucial conversations,Competitive strategy,Decision making,Influencing,Creativity / innovation,Collaboration,Cross-architecture,Effective listening,Story telling,...,Time management,Providing feedback,Team building,Communications (public speaking),Coaching,Social Media,Delegating,Finance,Productivity,Goal setting
22341,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
22342,1,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
22343,0,0,0,1,0,1,1,0,0,0,...,0,0,0,1,0,1,0,0,0,1
22344,1,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22345,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


### 1.1.2. Mentor Question 3: What are your personal interests?

This section creates a onehot encoded dataframe of the answers given by the mentors to this question.

The following responses (recorded in the "others" field) were merged as following:
* **My work. I know this sounds pleasing but it is the truth** -->  no match
* **Ranching** --> matches *Pets*
* **Car racing** --> matches *Extreme sports (sky diving)*
* **Creating technology video** --> matches *Tech hobbies* and *Photography*
* **Cycling** --> matches *Individual sports (golf, running)*
* **Automobile and Motorcycles collecting and building** --> matches *Puzzles & games*
* **yoga** --> matches *Individual sports (golf, running)*
* **Golf** --> matches *Individual sports (golf, running)*
* **Boating** --> matches *Individual sports (golf, running)*
* **Flying** --> matches *Travel*
* **Skiing and Scuba Diving** --> matches *Extreme sports (sky diving)* and *Individual sports (golf, running)*
* **karate** --> matches *Individual sports (golf, running)*
* **Diving** --> matches *Extreme sports (sky diving)*

In [692]:
df_interests_mentor = encode_onehot(df_mentor, pd.DataFrame([], index=df_mentor.index, columns=interests_mentor))

In [693]:
df_interests_mentor.columns

Index(['karate', 'Design', 'Boating',
       'Automobile and Motorcycles collecting and building',
       'Extreme sports (sky diving)', 'Meditation', 'Flying',
       'My work. I know this sounds pleasing but it is the truth', 'Dance',
       'Photography', 'Car racing', 'Skiing and Scuba Diving', 'Art',
       'Culture', 'Team sports (soccer, football)', 'Chess', 'Puzzles & games',
       'Shopping/fashion', 'Ranching', 'Creating technology video', 'Family',
       'Individual sports (golf, running)', 'Working out', 'Volunteering',
       'Religion', 'Politics', 'Tech hobbies', 'yoga', 'Golf',
       'Social networking', 'Travel', 'Wine tasting', 'Cycling', 'Gardening',
       'Music', 'Diving', 'Reading', 'Cooking', 'Pets'],
      dtype='object')

In [694]:
df_interests_mentor_final = pd.DataFrame()

# Pets & Ranching
cols = ["Pets", 
        "Ranching"] # own answer
df_interests_mentor_final = combine_cols(df_interests_mentor, cols, add_to_df=df_interests_mentor_final, combine_name="Pets")
exclude.append(cols)

# Extreme sports (sky diving) & Car racing & Skiing and Scuba Diving & Diving
cols = ["Extreme sports (sky diving)",
        "Car racing",
        "Skiing and Scuba Diving",# own answer
        "Diving"] # own answer
df_interests_mentor_final = combine_cols(df_interests_mentor, cols, add_to_df=df_interests_mentor_final, combine_name="Extreme sports (sky diving)")
exclude.append(cols)

# Tech Hobbies & Creating technology video
cols = ["Tech hobbies",
        "Creating technology video"] # own answer
df_interests_mentor_final = combine_cols(df_interests_mentor, cols, add_to_df=df_interests_mentor_final, combine_name="Tech Hobbies")
exclude.append(cols)

# Photography & Creating technology video &
cols = ["Photography",
        "Creating technology video"] # own answer
df_interests_mentor_final = combine_cols(df_interests_mentor, cols, add_to_df=df_interests_mentor_final, combine_name="Photography")
exclude.append(cols)

# Individual sports (golf, running) & Creating technology video &
cols = ["Individual sports (golf, running)",
        "Cycling", # own answer
        "yoga", # own answer
        "Golf", # own answer
        "Boating", # own answer
        "Skiing and Scuba Diving", # own answer
        "karate"] # own answer
df_interests_mentor_final = combine_cols(df_interests_mentor, cols, add_to_df=df_interests_mentor_final, combine_name="Individual sports (golf, running)")
exclude.append(cols)

# Puzzles & games & Automobile and Motorcycles collecting and building
cols = ["Puzzles & games",
        "Automobile and Motorcycles collecting and building"] # own answer
df_interests_mentor_final = combine_cols(df_interests_mentor, cols, add_to_df=df_interests_mentor_final, combine_name="Puzzles & games")
exclude.append(cols)

# Travel & Flying
cols = ["Travel",
        "Flying"] # own answer
df_interests_mentor_final = combine_cols(df_interests_mentor, cols, add_to_df=df_interests_mentor_final, combine_name="Travel")
exclude.append(cols)

exclude = [item for sublist in exclude for item in sublist]

In [695]:
for column in df_interests_mentor.columns: 
    if column not in exclude:
        df_interests_mentor_final[column] = list(df_interests_mentor[column])

In [696]:
df_interests_mentor_final.head()

,Pets,Extreme sports (sky diving),Tech Hobbies,Photography,"Individual sports (golf, running)",Puzzles & games,Travel,Design,Meditation,My work. I know this sounds pleasing but it is the truth,...,Working out,Volunteering,Religion,Politics,Social networking,Wine tasting,Gardening,Music,Reading,Cooking
22341,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
22342,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
22343,0,0,0,0,1,0,0,1,0,0,...,1,1,0,0,0,0,1,0,0,1
22344,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22345,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0


### 1.1.4. Combine all three mentor dataframes into one

The dataframes containing the Mentor's andwers "advice to give", "strengths" and "personal interests" are combined into one single one-hot encoded dataframe.

In [697]:
df_onehot_mentor = pd.concat([df_give_mentor_final, df_strengths_mentor_final,df_interests_mentor_final], axis=1)
df_onehot_mentor = df_onehot_mentor.reindex(sorted(df_onehot_mentor.columns), axis=1)

In [698]:
df_onehot_mentor.head()

,Accountability,Art,Change management,Chess,Coaching,Collaboration,Communications (public speaking),Competitive strategy,Cooking,Creativity / innovation,...,Transition career path,Travel,Volunteering,Wine tasting,Work / Life balance,Working out,day2day_advice,general_advice,own_advice,specific_advice
22341,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,1,1,1
22342,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,1,1,1,1
22343,1,0,0,0,0,1,1,0,1,1,...,0,0,1,0,1,1,1,1,1,1
22344,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,1,1,1,1
22345,0,1,0,0,1,0,0,0,0,1,...,0,0,1,0,0,0,1,1,1,1


## 1.2. Mentee data

Now we repeat the process with the Mentee data.

In [699]:
# read in mentee data
df_mentee_raw = pd.read_excel(r'FY21 - Mentee Interest - Chairman Club & Hall of Fame Mentoring Program.xlsx',sheet_name='From_a')
df_mentee_raw = pd.DataFrame(df_mentee_raw).set_index('Name', drop=True)
df_mentee_raw.shape

(78, 22)

In [700]:
df_mentee_raw = drop_nan_cols(df_mentee_raw)
df_mentee_raw.shape

# no columns had all NaNs

(78, 22)

In [701]:
df_mentee = df_mentee_raw[["What are you most likely to want out of this mentoring engagement?",
               "What are the strengths you would like to further develop?",
               "What are your personal interests?"]]

In [702]:
want_mentee = extract_choices(df_mentee["What are you most likely to want out of this mentoring engagement?"])
strengths_mentee = extract_choices(df_mentee["What are the strengths you would like to further develop?"])
interests_mentee = extract_choices(df_mentee["What are your personal interests?"])

### 1.2.1. Mentee Question 1: What are you most likely to want out of this mentoring engagement?

This section creates a onehot encoded dataframe of the answers given by the mentees to this question.

The responses were classifies into 4 main types: 
* **General Advice**
* **Specific Advice**
* **Day to Day Advice**
* **Own Advice**

The following responses (recorded in the "others" field) were edited in order to not skew the people matching. If a mentee chooses this, he/she will always be more different than each of the mentors, since no mentor will meet this creteria. 
* **Form a strategic plan on how to become the best (or the mindset that will help me do)**
* **situational experiences (wins and losses) Want to hear their perspective on how to be a good team player Want to hear their thoughts on market changes and where tech, energy and agriculture are headed.**

All responses were add to the **Own Advice** type, since otherwise the data would aswell be skewed. Now it has no impact on the mentee side of data.

In [703]:
df_want_mentee = encode_onehot(df_mentee, pd.DataFrame([], index=df_mentee.index, columns=want_mentee))

In [704]:
df_want_mentee.head()

,General strategic career advice,Form a strategic plan on how to become the best (or the mindset that will help me do),tactical day to day situations & problem solving,"Looking to hear about how they handle pressure, situational experiences (wins and losses) Want to hear their perspective on how to be a good team player Want to hear their thoughts on market changes and where tech, energy and agriculture are headed.",Specific career advice to prepare for next role,How to build better relationships with customers and partners /,Tactical day to day situations & problem solving,How one could maximize the talent and potential,Anything that they wish they had known in my situation,Know/uncover my strengths and differentiators
Name,,,,,,,,,,
12345,1,0,0,0,0,0,0,0,0,0
12346,1,0,0,0,0,0,1,0,0,0
12347,1,0,0,0,0,0,0,0,0,0
12348,1,0,0,0,0,0,0,0,0,0
12349,1,0,0,0,1,0,1,0,0,0


In [705]:
df_want_mentee_final = pd.DataFrame()
exclude = []

# combining the columns
cols = ["Form a strategic plan on how to become the best (or the mindset that will help me do)", 
        "General strategic career advice", 
        "Looking to hear about how they handle pressure, situational experiences (wins and losses) Want to hear their perspective on how to be a good team player Want to hear their thoughts on market changes and where tech, energy and agriculture are headed. "]
df_want_mentee_final = combine_cols(df_want_mentee, cols, add_to_df=df_want_mentee_final, combine_name="general_advice")
exclude.append(cols)

# Specific advice
cols = ["Form a strategic plan on how to become the best (or the mindset that will help me do)", # own answer
        "Specific career advice to prepare for next role", 
        "Looking to hear about how they handle pressure, situational experiences (wins and losses) Want to hear their perspective on how to be a good team player Want to hear their thoughts on market changes and where tech, energy and agriculture are headed. ",
        "How one could maximize the talent and potential",
        "How to build better relationships with customers and partners / "]
df_want_mentee_final = combine_cols(df_want_mentee, cols, add_to_df=df_want_mentee_final, combine_name="specific_advice")
exclude.append(cols)

# Advice of day to day situations & problem solving
cols = ["Form a strategic plan on how to become the best (or the mindset that will help me do)", # own answer
        "Tactical day to day situations & problem solving", 
        "tactical day to day situations & problem solving",
        "Looking to hear about how they handle pressure, situational experiences (wins and losses) Want to hear their perspective on how to be a good team player Want to hear their thoughts on market changes and where tech, energy and agriculture are headed. "]
df_want_mentee_final = combine_cols(df_want_mentee, cols, add_to_df=df_want_mentee_final, combine_name="day2day_advice")
exclude.append(cols)

# Own experience
cols = ["Form a strategic plan on how to become the best (or the mindset that will help me do)", 
        "General strategic career advice", 
        "Specific career advice to prepare for next role",
        "Tactical day to day situations & problem solving", 
        "tactical day to day situations & problem solving",
        "Looking to hear about how they handle pressure, situational experiences (wins and losses) Want to hear their perspective on how to be a good team player Want to hear their thoughts on market changes and where tech, energy and agriculture are headed. ",
        "Anything that they wish they had known in my situation",
        "Know/uncover my strengths and differentiators"]
df_want_mentee_final = combine_cols(df_want_mentee, cols, add_to_df=df_want_mentee_final, combine_name="own_advice")
exclude.append(cols)

exclude = [item for sublist in exclude for item in sublist]

In [706]:
for column in df_want_mentee.columns: 
    if column not in df_want_mentee_final.columns and column not in exclude:
        df_want_mentee_final[column] = list(df_want_mentee[column])

In [707]:
df_want_mentee_final.head()

,general_advice,specific_advice,day2day_advice,own_advice
12345,1,0,0,1
12346,1,0,1,1
12347,1,0,0,1
12348,1,0,0,1
12349,1,1,1,1


### 1.2.2. Mentee Question 2: What are the strengths you'd like to further develop?

This section creates a onehot encoded dataframe of the answers given by the mentees to this question.

In [708]:
# with ID column
df_strengths_mentee_final = encode_onehot(df_mentee, pd.DataFrame([], index=df_mentee.index, columns=strengths_mentee))

#df_strengths_mentee_final = encode_onehot(df_mentee, strengths_mentee)

In [709]:
df_strengths_mentee_final.head()

,Competitive strategy,Decision making,Influencing,Creativity / innovation,Effective listening,Cross-architecture,Collaboration,Story telling,Critical thinking,Work / Life balance,...,Team building,Crucial conversations,Communications (public speaking),Coaching,Social Media,Delegating,Finance,Productivity,Goal setting,Personal Brand
Name,,,,,,,,,,,,,,,,,,,,,
12345,0,1,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
12346,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12347,0,0,0,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
12348,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,1,1,0
12349,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0


### 1.2.3. Mentee Question 3: What are your personal interests?

This section creates a onehot encoded dataframe of the answers given by the mentees to this question.

The following responses (recorded in the "others" field) were merged as following:
* **Pen and Paper role-playing games (e.g. Dungeons & Dragons)** -->  matches *Puzzles & Games*
* **Programming (just beginning to learn!)** --> matches *Tech hobbies*
* **Coffee** --> matches *Cooking*

In [710]:
df_interests_mentee = encode_onehot(df_mentee, pd.DataFrame([], index=df_mentee.index, columns=interests_mentee))

In [711]:
df_interests_mentee.columns

Index(['Design', 'Psychology ', 'Extreme sports (sky diving)', 'Meditation',
       'Dance', 'Photography', 'Coffee', 'Art', 'Culture',
       'Team sports (soccer, football)', 'Chess', 'Shopping/fashion',
       'Puzzles & games', 'Family', 'Individual sports (golf, running)',
       'Working out', 'Volunteering', 'Politics', 'Religion', 'Tech hobbies',
       'Programming (just beginning to learn!)', 'Social networking', 'Travel',
       'Wine tasting', 'Stock market', 'Gardening', 'Music', 'Reading',
       'Pen and Paper role-playing games (e.g. Dungeons & Dragons)', 'Cooking',
       'Pets'],
      dtype='object')

In [712]:
df_interests_mentee_final = pd.DataFrame()
exclude = []

# Pets & Ranching
cols = ["Puzzles & games", 
        "Pen and Paper role-playing games (e.g. Dungeons & Dragons)"] # own answer
df_interests_mentee_final = combine_cols(df_interests_mentee, cols, add_to_df=df_interests_mentee_final, combine_name="Puzzles & games")
exclude.append(cols)

# Extreme sports (sky diving) & Car racing & Skiing and Scuba Diving & Diving
cols = ["Cooking",
        "Coffee"] # own answer
df_interests_mentee_final = combine_cols(df_interests_mentee, cols, add_to_df=df_interests_mentee_final, combine_name="Cooking")
exclude.append(cols)

# Tech Hobbies & Creating technology video
cols = ["Tech hobbies",
        "Programming (just beginning to learn!)"] # own answer
df_interests_mentee_final = combine_cols(df_interests_mentee, cols, add_to_df=df_interests_mentee_final, combine_name="Tech Hobbies")
exclude.append(cols)

# Stock market added to Finance strengths
for mentee in df_interests_mentee.index:
    try:
        if df_interests_mentee["Stock market"][mentee] == 1:
            df_strengths_mentee_final["Finance"][mentee] = 1
        exclude.append(["Stock market"])
    except:
        continue

# Stock market added to Influencing and Negotiating strengths
for mentee in df_interests_mentee.index:
    try:
        if df_interests_mentee["Psychology "].loc[mentee] == 1:
            df_strengths_mentee_final["Influencing"].loc[mentee] = 1
            df_strengths_mentee_final["Negotiating"].loc[mentee] = 1
        exclude.append(["Psychology "])
    except:
        continue

exclude = [item for sublist in exclude for item in sublist]

In [713]:
for column in df_interests_mentee.columns: 
    if column not in exclude:
        df_interests_mentee_final[column] = list(df_interests_mentee[column])

In [714]:
df_interests_mentee_final.columns

Index(['Puzzles & games', 'Cooking', 'Tech Hobbies', 'Design',
       'Extreme sports (sky diving)', 'Meditation', 'Dance', 'Photography',
       'Art', 'Culture', 'Team sports (soccer, football)', 'Chess',
       'Shopping/fashion', 'Family', 'Individual sports (golf, running)',
       'Working out', 'Volunteering', 'Politics', 'Religion',
       'Social networking', 'Travel', 'Wine tasting', 'Gardening', 'Music',
       'Reading', 'Pets'],
      dtype='object')

### 1.2.4. Combine all three mentee dataframes into one:

In [715]:
df_onehot_mentee = pd.concat([df_want_mentee_final, df_strengths_mentee_final, df_interests_mentee_final], axis=1)
df_onehot_mentee = df_onehot_mentee.reindex(sorted(df_onehot_mentee.columns), axis=1)

In [716]:
df_onehot_mentee.head()

,Accountability,Art,Change management,Chess,Coaching,Collaboration,Communications (public speaking),Competitive strategy,Cooking,Creativity / innovation,...,Transition career path,Travel,Volunteering,Wine tasting,Work / Life balance,Working out,day2day_advice,general_advice,own_advice,specific_advice
12345,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,1,1,0
12346,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,1,1,1,0
12347,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,1,0
12348,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,1,0,0,1,1,0
12349,0,1,0,0,0,0,1,0,0,0,...,0,1,0,0,0,1,1,1,1,1


## 1.3. Check the dataframes for errors

In [717]:
df_onehot_mentee.columns

Index(['Accountability', 'Art', 'Change management', 'Chess', 'Coaching',
       'Collaboration', 'Communications (public speaking)',
       'Competitive strategy', 'Cooking', 'Creativity / innovation',
       'Critical thinking', 'Cross-architecture', 'Crucial conversations',
       'Culture', 'Customer interaction', 'Dance', 'Decision making',
       'Delegating', 'Design', 'Design thinking', 'Effective listening',
       'Empowerment', 'Extreme sports (sky diving)', 'Family', 'Finance',
       'Gardening', 'Goal setting', 'Individual sports (golf, running)',
       'Influencing', 'Meditation', 'Music', 'Negotiating', 'Personal Brand',
       'Pets', 'Photography', 'Politics', 'Prioritization', 'Productivity',
       'Providing feedback', 'Puzzles & games', 'Reading', 'Recognition',
       'Religion', 'Shopping/fashion', 'Social Media', 'Social networking',
       'Story telling', 'Team building', 'Team sports (soccer, football)',
       'Tech Hobbies', 'Technical knowledge', 'Time m

In [718]:
df_onehot_mentor.columns

Index(['Accountability', 'Art', 'Change management', 'Chess', 'Coaching',
       'Collaboration', 'Communications (public speaking)',
       'Competitive strategy', 'Cooking', 'Creativity / innovation',
       'Critical thinking', 'Cross-architecture', 'Crucial conversations',
       'Culture', 'Customer interaction', 'Dance', 'Decision making',
       'Delegating', 'Design', 'Design thinking', 'Effective listening',
       'Empowerment', 'Extreme sports (sky diving)', 'Family', 'Finance',
       'Gardening', 'Goal setting', 'Individual sports (golf, running)',
       'Influencing', 'Meditation', 'Music',
       'My work. I know this sounds pleasing but it is the truth',
       'Negotiating', 'Personal Brand', 'Pets', 'Photography', 'Politics',
       'Prioritization', 'Productivity', 'Providing feedback',
       'Puzzles & games', 'Reading', 'Recognition', 'Religion',
       'Shopping/fashion', 'Social Media', 'Social networking',
       'Story telling', 'Team building', 'Team sports 

In [719]:
# same columns
for cols in df_onehot_mentee.columns:
    if cols not in df_onehot_mentor.columns:
        print(cols)

In [720]:
for cols in df_onehot_mentor.columns:
    if cols not in df_onehot_mentee.columns:
        print(cols)

My work. I know this sounds pleasing but it is the truth


In [721]:
# add missin columns
for column in df_onehot_mentee.columns:
    if column not in df_onehot_mentor.columns:
        df_onehot_mentor[column] = 0
        
for column in df_onehot_mentor.columns:
    if column not in df_onehot_mentee.columns:
        df_onehot_mentee[column] = 0

In [722]:
df_onehot_mentee

,Accountability,Art,Change management,Chess,Coaching,Collaboration,Communications (public speaking),Competitive strategy,Cooking,Creativity / innovation,...,Travel,Volunteering,Wine tasting,Work / Life balance,Working out,day2day_advice,general_advice,own_advice,specific_advice,My work. I know this sounds pleasing but it is the truth
12345,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,1,1,0,0
12346,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,1,1,1,1,0,0
12347,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,1,0,0
12348,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,1,1,0,0
12349,0,1,0,0,0,0,1,0,0,0,...,1,0,0,0,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12419,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
12420,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,1,1,1,1,0
12421,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,1,0,0
12423,0,0,1,0,0,0,1,1,0,0,...,1,0,1,0,0,0,1,1,0,0


# 2. Calculating Similarity of Users

In [723]:
# calculate the Jaccard distance between each mentee and each mentor
# 1-Jaccard distance = similarity between pairs
ary = scipy.spatial.distance.cdist(df_onehot_mentor.iloc[:,1:], df_onehot_mentee.iloc[:,1:], metric='jaccard')

people_similarity = pd.DataFrame(ary, columns=df_onehot_mentee.index.values, index=df_onehot_mentor.index.values)
people_similarity.head()

# mentees in columns (start 1), mentors in rows (start with 2)

,12345,12346,12347,12348,12349,12350,12351,12352,12353,12354,...,12414,12415,12416,12417,12418,12419,12420,12421,12423,12424
22341,0.789474,0.809524,0.909091,0.863636,0.833333,0.823529,0.736842,0.900000,0.793103,0.833333,...,0.705882,0.823529,0.882353,0.812500,0.941176,0.882353,0.777778,0.733333,0.809524,0.812500
22342,0.777778,0.800000,0.789474,0.857143,0.823529,0.882353,0.789474,0.857143,0.866667,0.823529,...,0.687500,0.642857,0.875000,0.875000,0.937500,0.875000,0.687500,0.615385,0.800000,0.714286
22343,0.800000,0.774194,0.878788,0.812500,0.740741,0.821429,0.843750,0.722222,0.628571,0.827586,...,0.793103,0.862069,0.857143,0.857143,0.851852,0.933333,0.750000,0.896552,0.812500,0.896552
22344,0.777778,0.736842,0.850000,0.909091,0.823529,0.944444,0.647059,0.857143,0.785714,0.823529,...,0.764706,0.812500,0.875000,0.800000,0.937500,0.875000,0.764706,0.800000,0.736842,0.875000
22345,0.875000,0.791667,0.782609,0.884615,0.809524,0.909091,0.782609,0.878788,0.741935,0.863636,...,0.869565,0.800000,0.850000,0.722222,0.952381,0.954545,0.700000,0.789474,0.791667,0.850000


In [724]:
# check which pair has the highest similarity in the dataset
a, b = people_similarity.stack().idxmin()
print(people_similarity.loc[[a], [b]])

          12398
22346  0.461538


In [725]:
people_similarity[12345].nsmallest(len(people_similarity))

22379    0.687500
22415    0.705882
22374    0.722222
22362    0.736842
22359    0.750000
           ...   
22407    0.944444
22370    0.944444
22371    0.944444
22360    0.944444
22364    1.000000
Name: 12345, Length: 85, dtype: float64

# 3. Assigning Mentor - Mentee Pairs

In [726]:
# function that assigns the list of mentors ranked by similarity to each mentee
def create_cluster(df):
    clusters = {}
    for mentee in df.columns:
        try:
            mentee_cluster = df[mentee].nsmallest(len(df))
            data = {mentee : list(mentee_cluster.index)}
            clusters.update(data)
        except:
            continue
    return clusters

In [727]:
people_cluster = create_cluster(people_similarity)
people_cluster

# for mentee 12345, the most similar mentor is 22342

{12345: [22379,
  22415,
  22374,
  22362,
  22359,
  22357,
  22395,
  22424,
  22422,
  22405,
  22404,
  22398,
  22389,
  22365,
  22423,
  22347,
  22344,
  22342,
  22353,
  22421,
  22381,
  22341,
  22416,
  22343,
  22397,
  22361,
  22400,
  22386,
  22392,
  22356,
  22426,
  22384,
  22406,
  22396,
  22354,
  22349,
  22414,
  22399,
  22373,
  22412,
  22408,
  22391,
  22401,
  22394,
  22410,
  22385,
  22355,
  22427,
  22351,
  22376,
  22375,
  22366,
  22346,
  22350,
  22383,
  22358,
  22425,
  22348,
  22352,
  22413,
  22417,
  22403,
  22393,
  22345,
  22380,
  22382,
  22369,
  22409,
  22390,
  22402,
  22411,
  22363,
  22418,
  22428,
  22419,
  22387,
  22367,
  22372,
  22368,
  22388,
  22407,
  22370,
  22371,
  22360,
  22364],
 12346: [22372,
  22394,
  22351,
  22348,
  22369,
  22349,
  22356,
  22357,
  22367,
  22404,
  22419,
  22346,
  22344,
  22410,
  22413,
  22425,
  22386,
  22411,
  22362,
  22354,
  22359,
  22343,
  22426,
  22373,
  22

# 4. Filter assignments by mentee/mentor preferences
We now have a ranking of the mentors that are most similar to the respective mentees. Each mentee has a list of the mentors ranked by their similarity regarding the survey choices.

Now we consider special preferences that the mentees/mentors chose within the survey.

The order of priorities considered is as following:

1. First preference of mentee 
2. Theatre preference of mentor
3. Role preference of mentor

In [728]:
# test to check if each mentee has at least 1 metor assigned to it
def test_assignment(info="default"):
    for mentee in people_cluster:
        if len(people_cluster[mentee]) < 1:
            raise ValueError(mentee, "has less than 1 mentor assigned! Info:", info)
    print("Test passed.")

## 4.1. Prepare Data: Theatre

### 4.1.1. Mentor: Theatre

In [729]:
df_mentor_theatre = df_mentor_raw[["Would you prefer a mentee in the same theatre?", 
                                "Your theatre?"]]

# filter by all the mentors that want a mentee in the same theatre
df_mentor_theatre_filter = df_mentor_theatre[df_mentor_theatre["Would you prefer a mentee in the same theatre?"]!="No preference"]

### 4.1.2. Mentee: Theatre

In [730]:
df_mentee_theatre = df_mentee_raw[["If possible, would you prefer a mentor in the same theatre?", 
                                   "Your theatre?"]]
df_mentee_theatre_filter = df_mentee_theatre[df_mentee_theatre["If possible, would you prefer a mentor in the same theatre?"]!="No preference"]

## 4.2. Prepare Data: Role
### 4.2.1. Mentor: Role

In [731]:
df_mentor_role = df_mentor_raw[["Would you prefer a mentee in a:", 
                                "What is your title?"]]

# filter by all the mentors that want a mentee in the same theatre
df_mentor_role_filter = df_mentor_role[df_mentor_role["Would you prefer a mentee in a:"]!="No preference"]

In [732]:
def replace_roles(df, col):
    technical = ["technical", "system", "se", "Technichal"] # systems atchitect
    sales = ["account manager", "business development", "client", "sales specialist", "financial", "partner", 
             "regional director", "sales manager", "renewal", "business solutions", "regional manager", "cx",
             "bdm", "product specialist", "Collaboration Executive", "Account Specialist", "vsam", "vpam"]
    for person in df.index:
        person_role = str((df[col].loc[person])).lower()

        for tech_role in technical:
            if tech_role.lower() in person_role:
                df[col][person] = "Technical role"
        for sales_role in sales:
            if sales_role.lower() in person_role:
                df[col][person] = "Sales role"

In [733]:
replace_roles(df_mentor_role, "What is your title?")

In [734]:
def test_roles(df, col):
    if len(set(df[col])) > 2:
        print(set(df[col]))
        raise ValueError("Error! Roles other than Sales & Technical Role present in dataset.")
    else:
        print("Test passed.")

In [735]:
test_roles(df_mentor_role, "What is your title?")

Test passed.


### 4.2.2. Mentee: Role

In [736]:
df_mentee_role = df_mentee_raw[["If possible, would you prefer a mentor in a:", 
                                "What is your title?"]]
df_mentee_role_filter = df_mentee_role[df_mentee_role["If possible, would you prefer a mentor in a:"]!="No preference"]

In [737]:
replace_roles(df_mentee_role, "What is your title?")

In [738]:
set(df_mentee_role["What is your title?"])

{'Sales role', 'Technical role'}

In [739]:
test_roles(df_mentee_role, "What is your title?")

Test passed.


## 4.3. Prepare Data: Segment
### 4.3.1 Mentor: Segment

In [740]:
df_mentor_segment = df_mentor_raw[["Your segment?"]]

for person in df_mentor_segment.index:
    if df_mentor_segment["Your segment?"][person] == "Enterprise Networking":
        df_mentor_segment["Your segment?"][person] = "Ent. Networking"

In [741]:
set(df_mentor_segment["Your segment?"])

{' Enterprise, commercial, public sector',
 'AppDynamics',
 'CGEM',
 'CGEM Enterprise',
 'CGEM Premier Global accounts',
 'CX',
 'Commercial',
 'Defence',
 'Ent. Networking',
 'Enterprise',
 'FED',
 'GES/CX',
 'GVSE',
 'GVSS',
 'Global Specialist Organization ',
 'Global Specialists Org',
 'Partner',
 'Public Sector',
 'Public Sector (Local and State Gov)',
 'Public Service',
 'Public sector',
 'SLED',
 'Service Provider',
 'Specialist, AppDynamics'}

### 4.3.2. Mentee: Segment

In [742]:
col = "If possible, which segment would you like your mentor to be from?"
df_mentee_segment = df_mentee_raw[[col]]
df_mentee_segment_filter = df_mentee_segment[df_mentee_segment[col]!="No preference"]
df_mentee_segment_filter = df_mentee_segment_filter[df_mentee_segment_filter[col]!="No Preference"]
df_mentee_segment_filter = df_mentee_segment_filter[df_mentee_segment_filter[col]!="Any"]
df_mentee_segment_filter = df_mentee_segment_filter[df_mentee_segment_filter[col]!="Enterprise, SP, Commercial, SLED, FED all work  "]

In [743]:
set(df_mentee_segment_filter["If possible, which segment would you like your mentor to be from?"])

{'CDA - Project Management',
 'Commercial',
 'DC',
 'Enterprise',
 'FED',
 'I still need to learn the above segments',
 'Partner',
 'SLED',
 'SP or CX',
 'Service Provider',
 'worked in multiple segments before'}

## 4.4. Filter by: First Priority Preference of Mentee

In [744]:
# prepare dataset
df_mentee_prio = df_mentee_raw[["If possible, what is your first priority in matching you with a mentor?"]]
df_mentee_prio = df_mentee_prio[df_mentee_prio["If possible, what is your first priority in matching you with a mentor?"]!="No preference"]
df_mentee_prio = df_mentee_prio[df_mentee_prio["If possible, what is your first priority in matching you with a mentor?"]!="Strengths you seek to develop"]
df_mentee_prio = df_mentee_prio[df_mentee_prio["If possible, what is your first priority in matching you with a mentor?"]!="Personal interests"]

In [745]:
def filter_by_pref(mentee, filter_by):
    if filter_by == "Theatre":
        mentee_theatre = df_mentee_theatre["Your theatre?"].loc[mentee]
        for mentor in df_mentor_theatre.index:
            mentor_theatre = df_mentor_theatre["Your theatre?"].loc[mentor] 
            if mentee_theatre != mentor_theatre:
                try:
                    people_cluster[mentee].remove(mentor)
                except:
                    continue
    elif filter_by == "Role":
        mentee_role = df_mentee_role["What is your title?"].loc[mentee]
        for mentor in df_mentor_role.index:
            mentor_role = df_mentor_role["What is your title?"].loc[mentor]
            if mentee_role != mentor_role:
                try:
                    people_cluster[mentee].remove(mentor)
                except:
                    continue
    elif filter_by == "Segment":
        if mentee in df_mentee_segment_filter.index:
            mentee_segment = df_mentee_segment_filter["If possible, which segment would you like your mentor to be from?"].loc[mentee]
            for mentor in df_mentor_segment.index:
                mentor_segment = df_mentor_segment["Your segment?"].loc[mentor]
                if mentee == "Morgan Carroll":
                    print(mentee)
                    print(mentor_segment)
                    if "sled" not in mentor_segment.lower() and "enterprise" not in mentor_segment.lower():
                        people_cluster[mentee].remove(mentor)
                elif mentee_segment.lower() not in mentor_segment.lower():
                    try:
                        people_cluster[mentee].remove(mentor)
                    except:
                        continue
    elif filter_by == "Gender":
        mentee_gender = df_mentee_raw["Gender"].loc[mentee]
        for mentor in df_mentor_raw.index:
            mentor_gender = df_mentor_raw["Gender"].loc[mentor]
            if mentee_gender != mentor_gender:
                try:
                    people_cluster[mentee].remove(mentor)
                except:
                    continue               

In [746]:
col ="If possible, what is your first priority in matching you with a mentor?"
for mentee in df_mentee_prio.index:
    try:
        if df_mentee_prio[col].loc[mentee] == "Theatre":
            filter_by_pref(mentee, "Theatre")
        elif df_mentee_prio[col].loc[mentee] == "Role (Tech/Sales)":
            filter_by_pref(mentee, "Role")
        elif df_mentee_prio[col].loc[mentee] == "Segment":
            filter_by_pref(mentee, "Segment")
        elif df_mentee_prio[col].loc[mentee] == "Gender":
            filter_by_pref(mentee, "Gender")
    except:
        continue

In [747]:
test_assignment()

Test passed.


In [748]:
#df_mentor_raw.isnull().sum(axis = 0)

# 4.5. Filter by Mentor Preference: Theatre

In [749]:
for mentor in df_mentor_raw.index:
    if df_mentor_raw["Would you prefer a mentee in the same theatre?"].loc[mentor]== "Yes":
        mentor_theatre = df_mentor_raw["Your theatre?"].loc[mentor]
        for mentee in people_cluster:
            mentee_theatre = df_mentee_raw["Your theatre?"].loc[mentee]
            if mentor_theatre != mentee_theatre:
                try:
                    people_cluster[mentee].remove(mentor)
                except:
                    continue         
# filter out mentees/mentor if both from AMER and APJC, because of timezone difference                   
for mentor in df_mentor_raw.index:
    mentor_theatre = df_mentor_raw["Your theatre?"].loc[mentor]
    for mentee in people_cluster:
        mentee_theatre = df_mentee_raw["Your theatre?"].loc[mentee]
        if (mentor_theatre == "AMER" and mentee_theatre == "APJC") or (mentor_theatre == "APJC" and mentee_theatre == "AMER"):
            try:
                people_cluster[mentee].remove(mentor)
            except:
                continue       

In [750]:
test_assignment()

Test passed.


In [751]:
for mentee in people_cluster:
    mentee_theatre = df_mentee_raw["Your theatre?"].loc[mentee]
    if mentee_theatre == "AMER":
        for mentor in people_cluster[mentee]:
            mentor_theatre = df_mentor_raw["Your theatre?"].loc[mentor]
            if mentor_theatre == "APJC":
                raise ValueError ("Oops, AMER mentee assigned with APJC mentor.")    
    elif mentee_theatre == "APJC":
        for mentor in people_cluster[mentee]:
            mentor_theatre = df_mentor_raw["Your theatre?"].loc[mentor]
            if mentor_theatre == "AMER":
                raise ValueError ("Oops, APJC mentee assigned with AMER mentor.")    
print("Test passed.")


Test passed.


In [752]:
people_cluster

{12345: [22415,
  22374,
  22359,
  22357,
  22395,
  22424,
  22422,
  22405,
  22404,
  22398,
  22389,
  22365,
  22423,
  22347,
  22342,
  22341,
  22416,
  22343,
  22397,
  22400,
  22356,
  22426,
  22406,
  22349,
  22414,
  22399,
  22373,
  22412,
  22408,
  22391,
  22401,
  22394,
  22410,
  22385,
  22427,
  22351,
  22376,
  22346,
  22350,
  22358,
  22425,
  22352,
  22417,
  22403,
  22393,
  22382,
  22369,
  22409,
  22402,
  22411,
  22363,
  22428,
  22419,
  22387,
  22372,
  22388,
  22371,
  22364],
 12346: [22394,
  22351,
  22356,
  22404,
  22344,
  22413,
  22411,
  22426,
  22406,
  22415,
  22427,
  22365,
  22352,
  22416,
  22341,
  22368,
  22376,
  22399,
  22405,
  22398,
  22374,
  22407,
  22421,
  22358,
  22364,
  22370],
 12347: [22410,
  22349,
  22387,
  22404,
  22398,
  22425,
  22374,
  22417,
  22400,
  22426,
  22414,
  22395,
  22422,
  22342,
  22372,
  22427,
  22347,
  22365,
  22350,
  22394,
  22351,
  22416,
  22411,
  22376,
  223

# 4.6. Filter by Mentor Preference: Role

In [753]:
for mentor in df_mentor_raw.index:
    if df_mentor_raw["Would you prefer a mentee in a:"].loc[mentor]!= "No preference":
        mentor_role = df_mentor_raw["Would you prefer a mentee in a:"].loc[mentor]
        for mentee in people_cluster:
            mentee_role = df_mentee_role["What is your title?"].loc[mentee]
            if mentor_role != mentee_role:
                try:
                    people_cluster[mentee].remove(mentor)
                except:
                    continue          

In [754]:
# for mentee in people_cluster:
#     print(len(people_cluster[mentee]))

In [755]:
test_assignment()

Test passed.


# 4.7. Filter by Mentor preference: Gender

In [756]:
for mentor in df_mentor_raw.index:
    if df_mentor_raw["Would you prefer a mentee of the same gender?"].loc[mentor]== "Yes":
        mentor_gender = df_mentor_raw["Gender"].loc[mentor]
        for mentee in people_cluster:
            mentee_gender = df_mentee_raw["Gender"].loc[mentee]
            if mentor_gender != mentee_gender:
                try:
                    people_cluster[mentee].remove(mentor)
                except:
                    continue          

In [757]:
test_assignment()

Test passed.


In [758]:
people_cluster

{12345: [22415,
  22374,
  22359,
  22395,
  22424,
  22422,
  22404,
  22389,
  22365,
  22423,
  22347,
  22342,
  22341,
  22416,
  22343,
  22397,
  22400,
  22356,
  22426,
  22406,
  22349,
  22414,
  22399,
  22373,
  22412,
  22401,
  22410,
  22385,
  22427,
  22351,
  22376,
  22346,
  22350,
  22358,
  22352,
  22417,
  22403,
  22393,
  22382,
  22369,
  22409,
  22402,
  22411,
  22363,
  22428,
  22419,
  22387,
  22372,
  22388,
  22371,
  22364],
 12346: [22351,
  22356,
  22404,
  22344,
  22413,
  22411,
  22426,
  22406,
  22415,
  22427,
  22365,
  22352,
  22416,
  22341,
  22376,
  22399,
  22374,
  22407,
  22358,
  22364],
 12347: [22410,
  22349,
  22387,
  22404,
  22374,
  22417,
  22400,
  22426,
  22395,
  22422,
  22342,
  22372,
  22427,
  22347,
  22365,
  22350,
  22351,
  22416,
  22411,
  22376,
  22359,
  22356,
  22393,
  22406,
  22363,
  22424,
  22373,
  22415,
  22401,
  22419,
  22385,
  22423,
  22389,
  22352,
  22358,
  22403,
  22397,
  224

# 4.8. Filter by Mentor specific preferences

In [759]:
for mentor in df_mentor_raw.index:
    if mentor == "22424":
        print(mentor)
        for mentee in people_cluster:
            mentee_country = df_mentee_raw["Your country of residence"].loc[mentee]
            print(mentee_country.lower())
            if mentee_country.lower() != "germany":
                try:
                    people_cluster[mentee].remove(mentor)
                    print("removed")
                except:
                    print("could not remove")
                    continue     
            else:
                print(people_cluster[mentee])

In [760]:
for mentee in people_cluster:
    if "22424" in people_cluster[mentee]:
        mentee_country = df_mentee_raw["Your country of residence"].loc[mentee]
        print(mentee_country)
        if not mentee_country.lower() == "germany":
            raise ValueError("Specif assignment failed.")
        else:
            print("Test passed.")

# 6. Filter if a Mentor has a Mentee in Mind

In [761]:
df_mentor_inmind = pd.DataFrame(df_mentor_raw["Let us know if you already have a CSAP mentee in mind"])

In [762]:
df_mentor_inmind = df_mentor_inmind[df_mentor_inmind['Let us know if you already have a CSAP mentee in mind'].notna()] 

In [763]:
pairs = {}
for mentee in df_mentee_raw.index:
    for answer in list(df_mentor_raw["Let us know if you already have a CSAP mentee in mind"]):
        if not pd.isnull(answer):
            if str(mentee) in str(answer):
                mentor = df_mentor_raw.index[df_mentor_raw["Let us know if you already have a CSAP mentee in mind"] == answer]
                pairs[mentee] = mentor[0]  

In [764]:
pairs

#mentor 22406 answered he/whe has mentee 12403 in mind

{}

Replace the metee in mind in the people cluster distionary:

In [765]:
for mentee_mind in pairs:
    mentor_mind = pairs[mentee_mind]
    for mentee in people_cluster:
        #print(mentee)
        if mentor_mind in people_cluster[mentee]:
            people_cluster[mentee].remove(mentor_mind)
        if mentee == mentee_mind:
            people_cluster[mentee].insert(0, mentor_mind)

In [766]:
test_assignment()

Test passed.


In [767]:
# test if only the mentee in "pairs" has the mentor in "pairs" assigned
def test_mentor_mind():
    for mentee in people_cluster:
        for mentor in people_cluster[mentee]:
            if mentor in pairs.values():
                if mentee not in pairs.keys():
                    print("Error for mentee:", mentee, "and mentor:", mentor)
    print("Test passed.")

In [768]:
test_mentor_mind()

Test passed.


In [769]:
people_cluster_unique = {}
for mentee in people_cluster.keys():
    people_cluster_unique[mentee] = people_cluster[mentee][0:10]

In [770]:
##### do not touch
def sort_unique(cluster, max_rounds=60, method="limited"):
    i = 0
    j = 0
    count = []
    for number in range(0,max_rounds):
        count.append(0)
    rounds = 0
    mentees = list(cluster.keys())
    while rounds < max_rounds:
        while i < len(cluster):
            while j < len(cluster):
                if j == i:
                    j+=1
                else:
                    to_compare = list(zip(cluster[mentees[i]], cluster[mentees[j]]))[0]
                    if to_compare[0] == to_compare[1]:
                        force_position = None
                        if mentees[i] in list(pairs.keys()):
                            force_position = 0
                        if mentees[j] in list(pairs.keys()):
                            force_position = 1
                        count[rounds] += 1
                        if force_position == None:
                            position = random.randint(0, 1)
                        else:
                            position = force_position
                            
                        if rounds > 1:
                            print(count[rounds-1])
                            if count[rounds-1] < count[rounds-2]:
                                #print("count gets smaller")
                                
                                if rounds % 5 == 0:
                                    if position == 0:
                                        try:
                                            cluster[mentees[j]][0], cluster[mentees[j]][2] = cluster[mentees[j]][2], cluster[mentees[j]][0]
                                            #print("1")
                                        except:
                                            continue
                                    else:
                                        try:
                                            cluster[mentees[i]][0], cluster[mentees[i]][2] = cluster[mentees[i]][2], cluster[mentees[i]][0]
                                            #print("2")          
                                        except:
                                            continue
                                else:
                                    if position == 0:
                                        try:
                                            cluster[mentees[j]][0], cluster[mentees[j]][1] = cluster[mentees[j]][1], cluster[mentees[j]][0]
                                            #print("3")
                                        except:
                                            continue
                                    else:
                                        try:
                                            cluster[mentees[i]][0], cluster[mentees[i]][1] = cluster[mentees[i]][1], cluster[mentees[i]][0]
                                            #print("4")
                                        except: 
                                            continue
                            else:
                                if method == "limited":
                                    if position == 0:
                                        try:
                                            cluster[mentees[j]][1], cluster[mentees[j]][3] = cluster[mentees[j]][3], cluster[mentees[j]][1]
                                            #print("5")
                                        except:
                                            continue
                                    else:
                                        try:    
                                            cluster[mentees[i]][1], cluster[mentees[i]][3] = cluster[mentees[i]][3], cluster[mentees[i]][1] 
                                            #print("6")
                                        except:
                                            continue
                                elif method == "medium":
                                    if position == 0:
                                        try:
                                            cluster[mentees[j]][2], cluster[mentees[j]][4] = cluster[mentees[j]][4], cluster[mentees[j]][2]
                                            #print("7")
                                        except:
                                            continue
                                    else:
                                        try:
                                            cluster[mentees[i]][2], cluster[mentees[i]][4] = cluster[mentees[i]][4], cluster[mentees[i]][2] 
                                            #print("8")
                                        except:
                                            continue                      
                                elif method == "hard": 
                                    indexes = [5,6,7,8,9,10]
                                    random_index = random.choice(indexes)
                                    print("Index:", random_index)
                                    if len(cluster[mentees[i]]) > random_index or len(cluster[mentees[j]]) > random_index:
                                        if position == 0:
                                            try:
                                                cluster[mentees[j]][3], cluster[mentees[j]][random_index] = cluster[mentees[j]][random_index], cluster[mentees[j]][3]
                                                #print("9")
                                            except:
                                                continue
                                        else:
                                            try:
                                                cluster[mentees[i]][3], cluster[mentees[i]][random_index] = cluster[mentees[i]][random_index], cluster[mentees[i]][3]  
                                                #print("11")
                                            except:
                                                continue
                                    else:
                                        if position == 0:
                                            cluster[mentees[j]][0], cluster[mentees[j]][-1] = cluster[mentees[j]][-1], cluster[mentees[j]][0]
                                            #print("13")
                                        else:
                                            cluster[mentees[i]][0], cluster[mentees[i]][-1] = cluster[mentees[i]][-1], cluster[mentees[i]][0] 
                                            #print("14")
                        else:
                            if position == 0:
                                cluster[mentees[j]][0], cluster[mentees[j]][1] = cluster[mentees[j]][1], cluster[mentees[j]][0]
                                #print("15")
                            else:
                                cluster[mentees[i]][0], cluster[mentees[i]][1] = cluster[mentees[i]][1], cluster[mentees[i]][0]   
                                #print("16")             
                    j+=1
            i += 1
            j = 0
        
        test = test_if_unique(cluster)
        if test == "Success":
            return "Success"
        else:
            if rounds == max_rounds-1:
                return "Failed"
            else:
                rounds += 1
                i = 0
                j = 0       
                
def test_if_unique(cluster):
    list_ranks = []
    for mentee in cluster:
        rank1 = cluster[mentee][0]
        list_ranks.append(rank1)
    list_ranks_set = set(list_ranks)
    list_ranks_set = list(list_ranks_set)
    list_ranks_set.sort()
    list_ranks.sort()
    if len(list_ranks) != len(list_ranks_set):
        return ValueError("Failed")
    else:
        return "Success"
    
##### do not touch

In [771]:
test_if_unique(people_cluster_unique)

ValueError('Failed')

In [772]:
# shuffle the assignments until the assignments on the first rank are unique

trys = 1
sorting = sort_unique(people_cluster_unique, method="limited")
trys += 1
while trys < 200:
    if sorting == "Success":
        print("Converged at try:", trys)
        trys = 100
        break
    elif trys < 10:
        sorting = sort_unique(people_cluster_unique, method="limited")
        trys += 1
    else:
        choice_list = ["medium"]*1 + ["hard"]*1 + ["limited"]*1
        random_method = random.choice(choice_list)
        print(random_method)
        sorting = sort_unique(people_cluster_unique, method=random_method)
        trys += 1 
if sorting == "Failed":
    print("Could not converge.")
test_assignment()   

73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
34
34
34
34
34
34
34
34
3

22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
31
31
31
31
31
31
31
31
31
31
31
31
31
31
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
25
25
25
2

14
14
14
14
14
14
14
14
14
14
17
17
17
17
17
17
17
17
17
17
17
17
17
17
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
28
28
28
28
28
28
28
28
28
28
28
28
28
28
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
23
23
23
23
23
23
23
23
23
23
23
23
23
23
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
1

18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
1

14
14
14
14
14
14
14
14
14
14
14
14
24
24
24
24
24
24
24
24
24
24
24
24
24
24
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
20
20
20
20
20
20
20
20
20
20
20
20
20
20
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
1

...

Index: 9
6
Index: 9
7
Index: 6
7
Index: 9
7
Index: 7
7
Index: 6
4
4
4
4
4
4
4
7
Index: 5
7
Index: 7
7
Index: 6
7
Index: 8
7
Index: 5
7
Index: 9
7
Index: 5
7
Index: 8
8
Index: 5
8
Index: 9
8
Index: 7
8
Index: 5
8
Index: 9
8
Index: 8
6
6
6
6
6
6
6
7
Index: 5
7
Index: 6
7
Index: 8
7
Index: 5
4
4
4
4
4
5
Index: 6
5
Index: 8
5
Index: 9
5
Index: 9
5
Index: 10
5
Index: 7
5
Index: 6
5
Index: 10
5
Index: 5
5
Index: 10
5
Index: 6
5
Index: 7
7
Index: 5
7
Index: 7
7
Index: 8
7
Index: 10
7
Index: 7
5
5
5
5
5
5
Index: 6
5
Index: 5
5
Index: 7
5
Index: 5
4
4
4
4
4
4
4
4
8
Index: 9
8
Index: 8
8
Index: 9
8
Index: 9
4
4
4
3
3
3
3
3
3
3
7
Index: 8
7
Index: 6
7
Index: 7
7
Index: 9
4
4
4
4
4
4
4
7
Index: 7
7
Index: 8
7
Index: 7
7
Index: 9
4
4
4
4
4
4
4
7
Index: 5
7
Index: 5
7
Index: 5
7
Index: 9
4
4
4
4
4
4
4
4
8
Index: 9
8
Index: 7
8
Index: 7
8
Index: 8
4
4
4
4
4
Index: 9
4
Index: 10
4
Index: 6
4
Index: 9
4
Index: 7
5
Index: 10
5
Index: 7
5
Index: 10
5
Index: 6
5
Index: 5
5
Index: 9
5
Index: 9
5
Index: 5
5

9
Index: 10
9
Index: 5
5
5
5
5
5
5
5
7
Index: 7
7
Index: 5
7
Index: 5
7
Index: 6
4
4
4
4
4
Index: 10
4
Index: 6
4
Index: 10
4
Index: 10
4
Index: 6
4
Index: 7
6
Index: 7
6
Index: 10
6
Index: 10
6
Index: 7
6
Index: 5
5
5
5
5
5
5
5
7
Index: 9
7
Index: 6
7
Index: 5
7
Index: 9
4
4
4
3
3
3
3
3
3
6
Index: 9
6
Index: 10
6
Index: 10
6
Index: 7
6
Index: 5
5
5
5
5
5
5
Index: 5
5
Index: 7
5
Index: 9
5
Index: 5
4
4
4
4
4
5
Index: 8
5
Index: 8
5
Index: 10
5
Index: 10
5
Index: 5
5
Index: 9
5
Index: 5
5
Index: 8
5
Index: 5
4
4
4
3
3
3
3
3
3
3
3
8
Index: 10
8
Index: 8
8
Index: 9
8
Index: 8
4
4
4
4
4
4
4
4
8
Index: 7
8
Index: 6
8
Index: 8
8
Index: 9
8
Index: 9
8
Index: 10
8
Index: 9
7
7
7
7
7
7
6
6
6
6
6
6
6
6
8
Index: 10
8
Index: 5
8
Index: 6
8
Index: 7
8
Index: 5
5
5
5
5
5
5
5
5
5
5
10
Index: 9
10
Index: 10
10
Index: 8
10
Index: 10
10
Index: 10
5
5
5
5
5
5
5
7
Index: 6
7
Index: 9
7
Index: 7
7
Index: 7
4
4
4
4
4
4
6
Index: 8
6
Index: 10
6
Index: 10
6
Index: 6
4
4
4
4
4
4
4
4
8
Index: 5
8
Index: 7
8
Ind

4
4
4
4
4
6
Index: 7
6
Index: 9
6
Index: 10
6
Index: 6
4
4
4
4
4
4
4
7
Index: 8
7
Index: 5
7
Index: 10
7
Index: 5
7
Index: 10
7
Index: 5
6
6
6
6
6
6
6
6
6
9
Index: 10
9
Index: 6
9
Index: 6
9
Index: 9
4
4
4
4
4
4
4
4
4
9
Index: 10
9
Index: 9
9
Index: 7
9
Index: 10
4
4
2
2
2
2
2
2
2
7
Index: 9
7
Index: 9
7
Index: 8
7
Index: 5
4
4
4
4
4
4
6
Index: 8
6
Index: 7
6
Index: 6
6
Index: 9
4
4
4
4
4
4
4
medium
9
9
9
9
4
4
4
4
4
4
6
6
6
6
4
4
4
4
4
5
5
5
5
4
4
4
4
4
4
4
7
7
7
7
4
4
4
4
4
4
4
4
4
4
10
10
10
10
4
4
4
4
4
4
4
4
8
8
8
8
4
4
4
4
4
4
4
4
4
9
9
9
9
4
4
4
3
3
3
3
3
3
3
7
7
7
7
4
4
4
4
4
4
4
7
7
7
7
4
4
4
4
4
4
4
4
8
8
8
8
4
4
4
4
4
4
4
7
7
7
7
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
hard
10
Index: 10
10
Index: 10
10
Index: 9
10
Index: 5
10
Index

6
Index: 9
3
3
3
3
3
3
3
7
Index: 10
7
Index: 9
7
Index: 7
7
Index: 5
7
Index: 10
7
Index: 6
6
6
6
6
6
6
6
Index: 10
6
Index: 8
6
Index: 8
6
Index: 10
6
Index: 5
6
Index: 8
6
Index: 8
6
Index: 7
6
Index: 5
6
Index: 10
6
Index: 6
5
5
5
5
5
5
5
7
Index: 7
7
Index: 6
7
Index: 10
7
Index: 6
7
Index: 8
5
5
5
5
5
5
5
7
Index: 10
7
Index: 6
7
Index: 9
7
Index: 10
7
Index: 7
7
Index: 9
6
6
6
6
6
5
5
5
5
5
5
Index: 6
5
Index: 10
5
Index: 10
5
Index: 6
5
Index: 6
5
Index: 10
5
Index: 7
5
Index: 5
5
Index: 6
4
4
4
4
4
4
6
Index: 7
6
Index: 10
6
Index: 10
6
Index: 5
6
Index: 8
6
Index: 7
6
Index: 8
7
Index: 7
7
Index: 7
7
Index: 8
7
Index: 9
7
Index: 8
5
5
5
3
3
3
3
3
3
3
7
Index: 8
7
Index: 6
7
Index: 10
7
Index: 9
7
Index: 6
5
5
5
5
4
4
4
4
4
4
4
4
8
Index: 9
8
Index: 7
8
Index: 5
8
Index: 6
8
Index: 9
8
Index: 9
6
6
6
6
6
6
6
6
8
Index: 7
8
Index: 10
8
Index: 9
8
Index: 10
8
Index: 5
8
Index: 10
8
Index: 6
8
Index: 7
8
Index: 8
8
Index: 8
8
Index: 7
8
Index: 8
4
4
4
4
4
4
4
7
Index: 10
7
Index:

8
Index: 6
8
Index: 9
8
Index: 7
8
Index: 5
5
5
5
5
5
5
5
7
Index: 9
7
Index: 10
7
Index: 10
7
Index: 7
7
Index: 5
7
Index: 8
6
6
6
6
6
6
6
6
8
Index: 5
8
Index: 8
8
Index: 6
8
Index: 8
4
4
4
4
4
5
Index: 10
5
Index: 7
5
Index: 10
5
Index: 5
5
Index: 8
5
Index: 9
5
Index: 9
5
Index: 7
5
Index: 10
4
4
4
3
3
3
3
Index: 7
3
Index: 5
2
2
2
3
Index: 6
3
Index: 10
3
Index: 10
3
Index: 8
3
Index: 7
2
2
2
Index: 6
2
Index: 7
2
Index: 10
2
Index: 9
2
Index: 8
3
Index: 10
3
Index: 10
3
Index: 10
3
Index: 5
4
Index: 10
4
Index: 5
2
2
2
Index: 10
2
Index: 10
2
Index: 7
3
Index: 9
3
Index: 7
2
2
2
2
4
Index: 8
4
Index: 9
2
2
2
3
Index: 10
3
Index: 5
2
2
2
2
4
Index: 6
4
Index: 7
2
2
2
2
2
5
Index: 5
5
Index: 6
2
2
2
3
Index: 8
3
Index: 5
2
2
2
2
2
5
Index: 6
5
Index: 10
2
2
2
2
2
2
6
Index: 5
6
Index: 7
2
2
2
3
Index: 7
3
Index: 10
2
2
2
2
4
Index: 5
4
Index: 6
2
2
2
Index: 8
2
Index: 7
2
Index: 10
2
Index: 5
2
Index: 7
3
Index: 6
3
Index: 7
2
2
2
2
2
5
Index: 6
5
Index: 10
5
Index: 5
5
Index: 5
4


In [773]:
test_if_unique(people_cluster_unique)

'Success'

# 7. Export Rankings as Excel Sheets

In [774]:
# Create an new Excel file and add a worksheet.
workbook = xlsxwriter.Workbook('Mentee_Mentor_AssignedbyRank_Anonym.xlsx')
worksheet = workbook.add_worksheet()

# format the sheet
worksheet.set_column('A:A', 30)
worksheet.set_column('B:K', 20)
# mentee cell (0,0)
mentee_cell = workbook.add_format({'bold': True})
mentee_cell.set_right()
mentee_cell.set_bottom()
# vertical
vertical = workbook.add_format({'bold': True})
vertical.set_right()
# horzontal
horizontal = workbook.add_format({'bold': True})
horizontal.set_bottom()

# Write "Mentee" cell
worksheet.write(0, 0, "Mentee", mentee_cell)


# Write mentee names
i = 1
for mentee in people_cluster_unique:
    worksheet.write(i, 0, str(mentee), vertical)
    i+=1

# Write ranks
i = 1
for j in range(0,10):
    worksheet.write(0, i, str("Rank " + str(i)), horizontal)
    i+=1

# Write mentor names
row = 1
for mentee in people_cluster_unique:
    mentors = people_cluster_unique[mentee]
    col = 1
    for mentor in mentors:
        worksheet.write(row, col, mentor)
        col+=1
    row +=1
    
workbook.close()

# 8. Test results

In [775]:
# function that tests whether a mentee has a mentor on rank 1 that is his most similar mentor
def rank_test():
    error_count = 0
    not_nr1_count = 0
    for mentee in people_similarity.columns:
        try:
            mentors_similar = list(people_similarity[mentee].nsmallest().index)
            mentors_assigned = people_cluster[mentee]
            for top_mentors in mentors_similar:
                if top_mentors not in mentors_similar:
                    error_count += 1
            if error_count != 0:
                raise ValueError("Error! Mentee", mentee, "has not all of its top 5 mentors in the final ranking.")
            if mentors_similar[0] != mentors_assigned[0]:
                not_nr1_count += 1
        except:
            continue
    print("Test passed.")
    print(not_nr1_count, "/", len(people_similarity.columns), "do not have mentor nr1 on rank1.")

In [776]:
rank_test()

Test passed.
47 / 78 do not have mentor nr1 on rank1.
